In [1]:
from bqplot import Figure, Axis, LinearScale, Scatter, Lines, CATEGORY10
from ipywidgets import HBox, VBox, Button, IntSlider, Play, jslink
from random import random, randint
#Student Exercise: Import 3 more modules

In [2]:
def norm_vector(x, y):
    """Normalize a vector."""
    length = math.sqrt(x**2 + y**2)
    return x/length, y/length

In [3]:
class Rectangle(Lines):
    """Simulate a Rectangle with a Lines mark containing 2 points."""
    
    def __init__(self, x_center, y_center, length=10, **kwargs):
        Lines.__init__(self, **kwargs)
        self.x_center, self.y_center = x_center, y_center
        self.length = length
        
        self.reset()
        
    def reset(self):
        self.x = [self.x_center - self.length//2, self.x_center + self.length//2 + 1]
        self.y = [self.y_center, self.y_center]
        
        # When a Brick is hit, it's not destroyed but just made visible=False
        # Thus resetting this here when the game is reset.
        self.visible = True
        
class Brick(Rectangle):
    def __init__(self, *args, **kwargs):
        Rectangle.__init__(self, *args, **kwargs)
        self.stroke_width = 20
        
        # Set a random color
        self.colors = [CATEGORY10[randint(0, len(CATEGORY10)-1)]]
        
class Racket(Rectangle):
    def __init__(self, *args, **kwargs):
        Rectangle.__init__(self, *args, **kwargs)
        self.stroke_width = 10
        
    def move(self, offset):
        # Racket only moves horizontally
        self.x = self.x[0]+offset, self.x[1]+offset

In [4]:
class Ball(Scatter):
    def __init__(self, **kwargs):
        Scatter.__init__(self, **kwargs)
        self.reset()
        self.dts = 0.1 #movement amplitude
        
    def reset(self):
        # ball is random placed at the bottom (between racket and bricks)
        self.x = np.array([randint(10, 90)]).astype('float64')
        self.y = np.array([randint(5, 10)]).astype('float64')
        
        # You can add a bit of acceleration if you want to spice things up!
        self.accel = np.array([0, 0]).astype('float64')
        
        # Compute a random (but upwward, not toward hte DEAD ZONE) initial speed direction
        # And make it speed = 10 in that direction!
        speedx, speedy = randint(5, 15), randint(5, 15)
        speedx, speedy = norm_vector(speedx, speedy)
        #speed up by a multiplier
        speedmultiplier = 10
        self.speed = np.array([speedmultiplier*speedx, speedmultiplier*speedy]).astype('float64')
        
    def move(self):
        newx, newy = self.x[0], self.y[0]
        
        self.speed += np.array(self.accel) * self.dts
        newx += self.speed[0] * self.dts
        newy += self.speed[1] * self.dts

        # Update the series
        with self.hold_sync():
            self.x, self.y = np.array([newx]), np.array([newy])
            
    def stop(self):
        self.x, self.y = 0,0

In [5]:
class Game(object):
    
    MSG_WELCOME = 'Click the Play button to start the game!'

    def __init__(self):
        self.fig = Figure(title=self.MSG_WELCOME)

        # The game area is a 100x100 square Figure
        # Only the bottom axis is particular: the DEAD ZONE in read.
        gamezone_size = 100
        sc = LinearScale(min=0, max=gamezone_size)
        self.scales = {"x": sc, "y": sc}
        ax_bottom = Axis(label='DEAD ZONE', scale=sc, num_ticks=0, color='red', label_color='red')
        ax_left = Axis(scale=sc, orientation='vertical', side='left', num_ticks=0, color='black')
        ax_top = Axis(scale=sc, orientation='horizontal', side='top', num_ticks=0, color='black')
        ax_right = Axis(scale=sc, orientation='vertical', side='right', num_ticks=0, color='black')
        self.fig.axes = [ax_bottom, ax_left, ax_right, ax_top]
        
        # Create bricks in a pyramid), a racket and a ball
        self.bricks = []
        self.bricks.extend([Brick(x, 30, scales=self.scales) for x in range(20, 85, 15)])
        self.bricks.extend([Brick(x, 50, scales=self.scales) for x in range(30, 80, 15)])
        self.bricks.extend([Brick(x, 70, scales=self.scales) for x in range(40, 75, 15)])
        self.bricks.extend([Brick(55, 90, scales=self.scales)])
        self.racket = Racket(50, 5, scales=self.scales)
        self.ball = Ball(scales=self.scales)

        all_marks = [self.racket, self.ball]
        all_marks.extend(self.bricks)
        self.fig.marks = all_marks
        
       # Callback called when the Play animation updates the invisible 'time_slider'
        def time_passing(change):
            newval = change['new']
            if newval == 0:
                # User clicked the STOP button so we reset the game
                self.reset()
            else:
                if newval == 1:
                    # User just started the game, update the label
                    self.fig.title = 'Good Luck!'
                self.ball.move()
                self.check_collisions()
                
        def create_ui(self):
            # To have ascyc updates (i.e. being able to move the ball continuously 
            # while having the user move the racket) I am leveraing the Play() animation
            # from ipywidgets. We link it to a IntSlider (time_slider) that we don't display.
            INFINITY = 1000000
            time_slider = IntSlider(min=0, max=INFINITY)
            time_slider.observe(time_passing, 'value')
            #self.but_play = Play(min=0, max=INFINITY, interval=40)
            #link = jslink((self.but_play, 'value'), (time_slider,'value'))

            # racket can be moved through these 2 buttons
            but_left = Button(description='LEFT')
            but_right = Button(description='RIGHT')
            but_left.on_click(lambda b: self.racket.move(-5 if self.racket.x[0] - 5 >= 0 else 0))
            but_right.on_click(lambda b: self.racket.move(5 if gamezone_size - self.racket.x[1] >=0  else 0))

            # The GUI of our game
            self.GUI = VBox([
                            HBox([time_, but_left, but_right]),
                            self.fig,
                            ])
        create_ui(self)
    
    def display(self):
        IPython.display.display(self.GUI)
    
    def reset(self):
        #self.fig.title = self.MSG_WELCOME
        [m.reset() for m in self.fig.marks]
    
    def check_collisions(self):
        x_min, x_max = self.scales['x'].min, self.scales['x'].max
        y_min, y_max = self.scales['y'].min, self.scales['y'].max

        newx, newy = self.ball.x[0], self.ball.y[0]

        # Collisions with walls (axes)
        if newx <= x_min:
            newx = x_min
            self.ball.speed[0] *= -1
            return

        if newx >= x_max:
            newx = x_max
            self.ball.speed[0] *= -1
            return
        
        if newy <= y_min:
            newy = y_min
            self.ball.speed[1] *= -1
            
            # Game over
            self.fig.title = 'GAME OVER! Play again!'
            
            #stop the ball
            self.but_play.value = self.but_play.max
            
            #reset the positions of bricks and ball
            self.reset()
            return
        
        #Student Execise: Implement the logic when the ball hits the top boundary
        
        # Collisions with Rectangles
        # Ignore those hit already (made visible=False)
        rect_list = [r for r in self.bricks+[self.racket] if r.visible]
        for rect in rect_list:
            # Since the ball position is its center, we account for the ball size
            # by adding a bit of offset around our rectangles to look like the
            # ball edge hit the obstacles. Below are empirically found.
            x_offset, y_offset = 1, 4
            
            # Collision happens if the ball is within the rectangle (widened with the offsets)
            x_check = (rect.x[0]-x_offset <= newx <= rect.x[1]+x_offset)
            y_check = (rect.y[0]-y_offset <= newy <= rect.y[1]+y_offset)
            if x_check and y_check:
                # Collision detected. Remove the Rectangle only if its a brick, not the racket!
                # We just make it non-visible so that we can reset it simply by making it visible.
                if rect != self.racket:
                    rect.visible = False

                # Try to best guess if it hit a top/bottom side or a left/right side
                # In order to know if we reverse the horizontal or vertical speed.
                if rect.x[0] <= newx <= rect.x[1]:
                    self.ball.speed[1] *= -1
                else:
                    self.ball.speed[0] *= -1
                    
                break
            
            # Update the ball position in case it was updated
            self.ball.x, self.ball.y = np.array([newx]), np.array([newy])

In [6]:
g = Game()
g.display()

NameError: name 'np' is not defined